In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/lab/TestSet-withLabels.xlsx /content/dataset/
!cp /content/drive/MyDrive/lab/TrainingSet.xlsx /content/dataset/

## Import datasets

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_excel('dataset/TrainingSet.xlsx', engine = 'openpyxl')
test = pd.read_excel('dataset/TestSet-withLabels.xlsx', engine = 'openpyxl')

In [ ]:
train.head()

,leaf.length,leaf.width,flower.length,flower.width,plant
0,5.4,3.7,1.5,0.2,Arctica
1,4.8,3.4,1.6,0.2,Arctica
2,4.8,3.0,1.4,0.1,Arctica
3,4.3,3.0,1.1,0.1,Arctica
4,5.8,4.0,1.2,0.2,Arctica


In [ ]:
test.head()

,leaf.length,leaf.width,flower.length,flower.width,plant
0,4.4,2.9,1.4,0.2,Arctica
1,4.6,3.1,1.5,0.2,Arctica
2,4.6,3.4,1.4,0.3,Arctica
3,4.7,3.2,1.3,0.2,Arctica
4,4.9,3.0,1.4,0.2,Arctica


In [ ]:
len(train.index)

120

## Processing datasets

Includes standard scaling of numerical values and one hot encoding of target variable (plant type).

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
plant_num = train.drop("plant", axis=1)
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])
plant_num_tr = num_pipeline.fit_transform(plant_num)

In [ ]:
num_attribs = list(plant_num)
cat_attribs = ["plant"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])
plant_prepared = full_pipeline.fit_transform(train)
plant_X = plant_prepared[:, 0:4].astype(float)
plant_Y = plant_prepared[:, 4:]

## Creating Neural Network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential(
[
    layers.Dense(30, activation ="relu"),
    layers.Dense(30, activation ="relu"),
    layers.Dense(30, activation ="relu"),
    layers.Dense(30, activation ="relu"),
    layers.Dense(3, activation = "softmax")    
])

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"]) #try changing optimizers

In [ ]:
model.fit(plant_X, plant_Y, epochs=20, validation_split=0.2, batch_size=15)

Epoch 1/20
7/7 [==============================] - 1s 28ms/step - loss: 1.0948 - accuracy: 0.2083 - val_loss: 1.0023 - val_accuracy: 0.5417
Epoch 2/20
7/7 [==============================] - 0s 5ms/step - loss: 1.0111 - accuracy: 0.6771 - val_loss: 0.9632 - val_accuracy: 0.5417
Epoch 3/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9223 - accuracy: 0.8958 - val_loss: 0.9016 - val_accuracy: 0.5417
Epoch 4/20
7/7 [==============================] - 0s 6ms/step - loss: 0.8275 - accuracy: 0.8958 - val_loss: 0.8660 - val_accuracy: 0.4583
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 0.7223 - accuracy: 0.9167 - val_loss: 0.8078 - val_accuracy: 0.5417
Epoch 6/20
7/7 [==============================] - 0s 5ms/step - loss: 0.6104 - accuracy: 0.9167 - val_loss: 0.7430 - val_accuracy: 0.5833
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 0.5093 - accuracy: 0.9167 - val_loss: 0.7303 - val_accuracy: 0.5833
Epoch 8/20
7/7 [=================

## Process Test set before evaluating model

In [ ]:
plant_test = full_pipeline.transform(test)

plant_test_X = plant_test[:, 0:4].astype(float)
plant_test_Y = plant_test[:, 4:]

model.evaluate(plant_test_X, plant_test_Y)

1/1 [==============================] - 0s 15ms/step - loss: 0.2208 - accuracy: 0.9667


[0.22079992294311523, 0.9666666388511658]

In [ ]:
predictions = model.predict(plant_test_X)
predictions

array([[9.9843627e-01, 6.1664107e-04, 9.4704417e-04],
       [9.9882835e-01, 5.1229651e-04, 6.5936375e-04],
       [9.9942517e-01, 2.6918703e-04, 3.0566542e-04],
       [9.9915016e-01, 3.8450584e-04, 4.6536259e-04],
       [9.9750358e-01, 1.0354965e-03, 1.4609876e-03],
       [9.9839741e-01, 6.9472310e-04, 9.0787240e-04],
       [1.0430496e-02, 3.7340265e-02, 9.5222920e-01],
       [2.9122152e-03, 1.2957321e-01, 8.6751455e-01],
       [9.9938881e-01, 2.7597006e-04, 3.3523492e-04],
       [9.9893171e-01, 4.8246823e-04, 5.8576383e-04],
       [9.9906701e-01, 4.1784736e-04, 5.1521783e-04],
       [1.1546876e-02, 7.7842399e-02, 9.1061074e-01],
       [9.9871337e-01, 4.7573028e-04, 8.1093714e-04],
       [6.4130273e-04, 5.7354778e-02, 9.4200391e-01],
       [8.2690213e-03, 9.8502435e-02, 8.9322859e-01],
       [1.8558435e-03, 5.7761359e-01, 4.2053056e-01],
       [1.3873868e-02, 4.1099134e-01, 5.7513481e-01],
       [2.3602064e-04, 9.6914434e-01, 3.0619631e-02],
       [1.2727907e-03, 6.403

In [ ]:
predictions = model.predict_classes(plant_test_X, verbose = 1)
predictions

1/1 [==============================] - 0s 12ms/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 2, 1, 1, 2, 2, 1,
       2, 1, 2, 2, 1, 1, 1, 1])

In [ ]:
df = pd.DataFrame(predictions, columns = ['plant_pred']).replace({0:'Artica', 1:'Carolinian', 2:'Harlequin'})
df.join(test['plant'])

,plant_pred,plant
0,Artica,Arctica
1,Artica,Arctica
2,Artica,Arctica
3,Artica,Arctica
4,Artica,Arctica
5,Artica,Arctica
6,Harlequin,Harlequin
7,Harlequin,Carolinian
8,Artica,Arctica
9,Artica,Arctica
